# Goldratt's Dice Game

*Paul Bayer, 2017-07-24*

A classical illustration that dependencies and statistical fluctuations diminish the througput through a system is Goldratt's Dice Game from his business novel "The Goal".

Alex Rogo, the hero of the novel plays a game with five boys:

>*While they get go get the others, I figure out the details. The system I've set up is intended to "process" matches. It does this by moveng a quatity of match sticks out of their box, and through each of the bowls in succession. The dice determine how many matches can be moved from one bowl to the next. The dice represent the capacity of each resource, each bowl; the set of bowls are my dependent events, my stages of production. Each has exactly the same capacity as the others, but its actual yield will fluctuate somewhat.*

>*In order to keep those fluctuations minimal, however, I decide to use only one of the dice. This allows the fluctiations to range from one to six. So from the first bowl, I can move to the next bowls in line any quantity of matches ranging from a minimum of one to a maximum of six.*

>*Throughput in thos system is the speed at which matches come out of the last bowl, Inventory consists of the total number of matches in all of the bowls at any time. And I'm going to assume that market demand is exactly equal to the average number of matches that the system can process. Production capacity of each resource and market demand are perfectly in balance. So that means I now have a model of a perfectly balanced manufactoring plant.*

>*Five of the boys decide to play. Besides Dave, there are Andy, Ben, Chuck, and Evan. Each of them sits behind one of the bowls. I find some paer and a pencil to record what happens. Then I explain what they're supposed to do.*

>*"The idea is to move as many matches as you can from your bowl to the bowl on your right. When it's your turn, you roll the die, and the number that comes up is the number of matches you can move. Got it?"*

>*They all nod. "But you can only move as many matches as you've got in your bowl. So if you roll a five and you only have two matches in your bowl, then you can only move two matches. And if it comes to your turn and you don't have any matches, then naturally you can't move any."*

>*Eliyahu M Goldratt: The Goal,– 3rd ed, p. 105*

Then Rogo explains to the boys that with the die on average they should pass ```3.5```matches through the system, so after twenty cycles they should have got an output of seventy.

## A Simulation

Since we don't have enough boys as did Rogo, we have to do a simulation. But – as Julia is fast – we can do easily ```100``` cycles. And – according to the story – we should expect an output of ```350```matches. Let's do it.

In [2]:
using SimJulia, Distributions

function boy(sim::Simulation, nr::Int64, bowls::Dict{Int64,Int64})
  while true
    yield(Timeout(sim, 1))
    mystock = bowls[nr]
    roll = rand(DiscreteUniform(1, 6))
    matches = min(roll, mystock)
    bowls[nr]   -= matches
    bowls[nr+1] += matches
  end
end

# logging functions
function logINV(time::Float64, bowls::Dict{Int64,Int64}, tp::Int64)
  print(@sprintf("%5.1f: INV: ", now(sim)))
  for i in 1:5
    print(@sprintf("%3d - ", bowls[i]))
  end
  println(@sprintf("%4d -- TP: %d", bowls[6], tp))
end

function clog(sim::Simulation, bowls::Dict{Int64,Int64})
  while true
    outp = bowls[6]
    yield(Timeout(sim, 1))
    logINV(now(sim), bowls, bowls[6]-outp)
  end
end

# initalization
srand(1234)       # seed random number generator for reproducibility
bowls = Dict{Int64,Int64}(1=>1e3-1, 2=>0, 3=>0, 4=>0, 5=>0, 6=>0)  # initialize bowls with matches

sim = Simulation()
for i = 1:5
  @process boy(sim, i, bowls)  # create 5 players
end
@process clog(sim, bowls)         # add logging after each run

println("time       ANDY   BEN  CHUCK  DAVE  EVAN  finished")
logINV(0.0, bowls, 0)
run(sim, 101)


time       ANDY   BEN  CHUCK  DAVE  EVAN  finished
  0.0: INV: 999 -   0 -   0 -   0 -   0 -    0 -- TP: 0
  1.0: INV: 994 -   0 -   3 -   0 -   0 -    2 -- TP: 2
  2.0: INV: 991 -   0 -   0 -   3 -   0 -    5 -- TP: 3
  3.0: INV: 987 -   2 -   0 -   2 -   1 -    7 -- TP: 2
  4.0: INV: 986 -   0 -   0 -   0 -   4 -    9 -- TP: 2
  5.0: INV: 985 -   0 -   0 -   0 -   1 -   13 -- TP: 4
  6.0: INV: 981 -   0 -   0 -   2 -   0 -   16 -- TP: 3
  7.0: INV: 977 -   3 -   0 -   0 -   2 -   17 -- TP: 1
  8.0: INV: 972 -   3 -   0 -   3 -   0 -   21 -- TP: 4
  9.0: INV: 968 -   5 -   0 -   1 -   2 -   23 -- TP: 2
 10.0: INV: 962 -   8 -   0 -   0 -   4 -   25 -- TP: 2
 11.0: INV: 956 -   8 -   2 -   2 -   4 -   27 -- TP: 2
 12.0: INV: 951 -   8 -   1 -   5 -   5 -   29 -- TP: 2
 13.0: INV: 945 -   8 -   2 -   6 -   5 -   33 -- TP: 4
 14.0: INV: 942 -   6 -   3 -   7 -   7 -   34 -- TP: 1
 15.0: INV: 939 -   4 -   6 -   6 -   8 -   36 -- TP: 2
 16.0: INV: 934 -   7 -   4 -   7 -  10 -   37 -- TP:

After 100 runs we got an output of 314 and therefore an average throughput of 3.14, much less than expected. What went wrong? We had an unlimited supply of matches, a perfectly balanced line, committed players, why didn't they succeed to produce the expected outcome but much less so?

Here naturally occuring fluctuations in outcomes of die rolls lead to the fact that not enough matches can be passed on in the system for further processing. So even if players down the line roll good numbers, they cannot pass on matches since they got none. Inventory builds up in the system.

## Kanban

Against too much inventory we have Kanban. So let's set the bowls of boys 2 … 5 to a maximum number – of say ```10``` – and do it again. Therefore we have to modify our boy function:   

In [3]:
const MAXM = 10

function boy(sim::Simulation, nr::Int64, bowls::Dict{Int64,Int64})
  while true
    yield(Timeout(sim, 1))
    mystock = bowls[nr]
    roll = rand(DiscreteUniform(1, 6))
    if nr < 5
      matches = min(roll, mystock, MAXM-bowls[nr+1])
    else
      matches = min(roll, mystock)
    end
    bowls[nr]   -= matches
    bowls[nr+1] += matches
  end
end

# initalization
srand(1234)       # seed random number generator for reproducibility
bowls = Dict{Int64,Int64}(1=>1e3-1, 2=>0, 3=>0, 4=>0, 5=>0, 6=>0)  # initialize bowls with matches

sim = Simulation()
for i = 1:5
  @process boy(sim, i, bowls)  # create 5 players
end
@process clog(sim, bowls)         # add logging after each run

println("time       ANDY   BEN  CHUCK  DAVE  EVAN  finished")
logINV(0.0, bowls, 0)
run(sim, 101)


time       ANDY   BEN  CHUCK  DAVE  EVAN  finished
  0.0: INV: 999 -   0 -   0 -   0 -   0 -    0 -- TP: 0
  1.0: INV: 994 -   0 -   3 -   0 -   0 -    2 -- TP: 2
  2.0: INV: 991 -   0 -   0 -   3 -   0 -    5 -- TP: 3
  3.0: INV: 987 -   2 -   0 -   2 -   1 -    7 -- TP: 2
  4.0: INV: 986 -   0 -   0 -   0 -   4 -    9 -- TP: 2
  5.0: INV: 985 -   0 -   0 -   0 -   1 -   13 -- TP: 4
  6.0: INV: 981 -   0 -   0 -   2 -   0 -   16 -- TP: 3
  7.0: INV: 977 -   3 -   0 -   0 -   2 -   17 -- TP: 1
  8.0: INV: 972 -   3 -   0 -   3 -   0 -   21 -- TP: 4
  9.0: INV: 968 -   5 -   0 -   1 -   2 -   23 -- TP: 2
 10.0: INV: 963 -   7 -   0 -   0 -   4 -   25 -- TP: 2
 11.0: INV: 960 -   4 -   2 -   2 -   4 -   27 -- TP: 2
 12.0: INV: 955 -   4 -   1 -   5 -   5 -   29 -- TP: 2
 13.0: INV: 949 -   4 -   2 -   6 -   5 -   33 -- TP: 4
 14.0: INV: 946 -   2 -   3 -   7 -   7 -   34 -- TP: 1
 15.0: INV: 943 -   0 -   6 -   6 -   8 -   36 -- TP: 2
 16.0: INV: 938 -   3 -   4 -   8 -   9 -   37 -- TP:

Uups! We throttled our system further, to an output of ```289``` and a throughput of ```2.89``` matches per cycle. But we got much less inventory in the system. So seemingly Kanban is no solution for our throughput problem but constrains the system further.

## Conclusion

With the Dice Game – after variation [1] – we introduced one further parameter into our mini-experiments with system performance: **dependencies**. 

- If we elongated our line further, to 10 or 50 stations, output would be further reduced.
- If we reduced our in-process-inventories further we throttle the system all the more.

We have to get a grip on those effects.

-------------------

[1]: In the Post Office we had to tackle with the variation in arrival rates of customers. Now we have no variation in input, only in-process-variation.